In [1]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
import mindpype as mp
import pkgutil, numpy as np

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline.

In [3]:
# Create the pipeline using the factory method
session = mp.Session.create()

Following the creation of the pipeline, we need to create the graphs, which are the representations of the pipeline in MindPype.

Each distinct pipeline would require a different graph object. For this tutorial, we will only require a single graph.

In [4]:
# Create the graph using the factory method
graph = mp.Graph.create(session)

With a graph created, we can now attach our input and output data containers. Since the input data is known, we can create a tensor (the data container in MindPype), using the ```create_from_data()``` factory method. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.       

In [5]:
# Create a random input tensor using the factory method
input_tensor = mp.Tensor.create_from_data(session, data=np.random.rand(10, 10))

# Create an output tensor using the factory method
output_tensor = mp.Tensor.create(session, shape=(10, 10))

With the input and output tensors, we can know add the processing nodes to the pipeline. For this example, we are using a single processing step, or kernel; the baseline correction kernel. We can use the following factory method to add the kernel to a node, and add the node to the graph we created previously.

Futhermore, we must pass the input and output tensors, as well as the required parameters for the kernel (in this case, the baseline period), to the factory method shown below.

In [6]:
# Create the baseline node, passing the graph, input tensor, output tensor, and baseline period
baseline_node = mp.kernels.BaselineCorrectionKernel.add_to_graph(graph, input_tensor, output_tensor, baseline_period = [0, 10])

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

In [7]:
# Verify the structure and inputs/outputs of the graph
graph.verify()


Verifying kernel BaselineCorrection...


Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.

In [8]:
# Initialize the graph - optional for this pipeline
graph.initialize()

The graph is now ready to execute the provided input data. For other paradigms, we may opt to execute the graph in a loop. If we were using class-separated data, we would need to pass a task label to poll the correct task.

In [9]:
# Execute the graph
graph.execute()



Executing trial with label: None


In [10]:
# Visualize the input and output data
print(input_tensor.data)
print(output_tensor.data)

[[7.34110771e-01 4.57722669e-01 6.66531872e-01 7.11005230e-01
  7.92314656e-01 5.17775675e-01 1.81822919e-01 5.78707113e-01
  4.67047322e-04 4.97898212e-01]
 [1.95414557e-01 4.14465986e-02 7.61692333e-01 9.18318212e-01
  1.65679925e-01 8.15532606e-01 8.42312864e-01 1.29486755e-02
  2.40723641e-01 9.20204461e-01]
 [8.64104599e-01 3.26856007e-01 9.27892817e-01 6.80225847e-01
  6.52112772e-01 5.53260594e-01 5.27720072e-01 8.54729520e-01
  1.01104138e-02 2.89426484e-01]
 [4.60970760e-01 5.93051275e-01 2.15962281e-01 2.28389197e-01
  2.71217301e-01 2.42680762e-01 4.90356274e-01 4.24047406e-01
  9.95146262e-01 8.79795977e-01]
 [7.60716223e-02 2.79496842e-01 9.66411182e-01 1.91701123e-01
  2.07176762e-01 2.94752828e-01 7.80670916e-01 6.26475926e-01
  9.60608542e-01 9.83592492e-01]
 [9.53101755e-01 4.79378256e-01 7.70374397e-01 1.67977667e-02
  8.40488551e-01 8.29145311e-01 4.54152528e-01 3.01784268e-01
  7.91131372e-01 6.42619367e-01]
 [6.27576194e-01 8.88907700e-01 2.01042348e-01 4.61537560e

In [11]:
# Confirm proper functionality
corrected = input_tensor.data - np.mean(input_tensor.data, axis = -1, keepdims = True)
assert np.allclose(output_tensor.data, corrected)

[[ 0.00394006 -0.3320127   0.0648715  -0.51336857 -0.0159374 ]
 [ 0.32410522  0.35088548 -0.47847871 -0.25070375  0.42877707]
 [-0.01538332 -0.04092384  0.28608561 -0.5585335  -0.27921743]
 [-0.23748099  0.01019452 -0.05611434  0.51498451  0.39963423]
 [-0.241943    0.24397509  0.0897801   0.42391272  0.44689667]
 [ 0.22124795 -0.15374483 -0.30611309  0.18323402  0.03472201]
 [-0.19894892  0.35493104 -0.30903483 -0.13319615 -0.11433924]
 [ 0.14523086  0.04765389  0.0023633  -0.08306825 -0.23557747]
 [-0.21215894  0.04746841 -0.21664157 -0.18882907  0.24427325]
 [-0.09649057  0.12595255 -0.175553    0.27600883  0.08278516]]
[[ 0.00394006 -0.3320127   0.0648715  -0.51336857 -0.0159374 ]
 [ 0.32410522  0.35088548 -0.47847871 -0.25070375  0.42877707]
 [-0.01538332 -0.04092384  0.28608561 -0.5585335  -0.27921743]
 [-0.23748099  0.01019452 -0.05611434  0.51498451  0.39963423]
 [-0.241943    0.24397509  0.0897801   0.42391272  0.44689667]
 [ 0.22124795 -0.15374483 -0.30611309  0.18323402  0.0